In [1]:
from pathlib import Path
import itertools
from matplotlib import pyplot as plt
import pandas as pd
import re

In [2]:
# Set path to folder with faces: stim_data_dir
stim_data_dir = Path("...", "data", "Nimstim faces")

In [13]:
def create_csv(path=stim_data_dir, write=False):
    """ Create dataframe with all image IDs and sex, emotion, and mouth position depicted on image"""

    # Read all images
    files_path = list(path.glob('*.bmp'))
    files = list(x.stem for x in files_path)

    # Create codebook with the meaning of all abbreviations
    codebook = {0: {'female', 'male'},
           1: {'calm', 'angry', 'happy', 'fear', 'sad', 'surprised', 'neutral', 'disgust'},
           2: {'open', 'closed', 'exuberant'}}
    output = [[], [], []]

    # Decode sex, emotion, and mouth position depicted on image
    for file in files:
        file_parts = file.split('_')
        file_parts[0] = file_parts[0][-1]

        for i in range(0, 3):
            try:
                code = re.findall('(?<=\')' + file_parts[i].lower() + '\w*', str(codebook[i]))[0]
                output[i].append(code)
            except:
                if file_parts[i].lower() == 'sp':
                    output[i].append('surprised')
                elif file_parts[i].lower() == 'x':
                    output[i].append('exuberant')
                else:
                    next

    # Create dataframe with all collected info about the image
    df = {'file': files_path,
          'sex': output[0],
          'emotion': output[1],
          'mouth': output[2]}
    df = pd.DataFrame(df)
    df.index.name = 'picture_id'
    
    # Write dataframe to csv file
    if write:
        df.to_csv('overview.csv', index=True)
    
    return df


In [14]:
# Create overview of all images with their corresponding sex, emotion and mouth positioning
overview = create_csv(write=True)
overview

,file,sex,emotion,mouth
picture_id,,,,
0,data\Nimstim faces\01F_AN_C.bmp,female,angry,closed
1,data\Nimstim faces\01F_AN_O.bmp,female,angry,open
2,data\Nimstim faces\01F_CA_C.bmp,female,calm,closed
3,data\Nimstim faces\01F_CA_O.bmp,female,calm,open
4,data\Nimstim faces\01F_DI_C.bmp,female,disgust,closed
...,...,...,...,...
652,data\Nimstim faces\43M_NE_C.BMP,male,neutral,closed
653,data\Nimstim faces\43M_NE_O.BMP,male,neutral,open
654,data\Nimstim faces\43M_SA_C.BMP,male,sad,closed


In [5]:
from protosc.preprocessing import GreyScale, ViolaJones, CutCircle
from protosc.pipe_complex import PipeComplex
from protosc.feature_extraction import FourierFeatures
from protosc.io import ReadImage
grayscale = True

In [6]:
# Define pipeline
pipe1 = ReadImage()*ViolaJones(20)*CutCircle()*FourierFeatures()
pipe2 = ReadImage()*GreyScale()*ViolaJones(20)*CutCircle()*FourierFeatures()

pipe_complex = pipe1 + pipe2
print(pipe_complex)

Pipe: ReadImage -> ViolaJones_20 -> CutCircle -> FourierFeatures_a8s7
Pipe: ReadImage -> GreyScale -> ViolaJones_20 -> CutCircle -> FourierFeatures_a8s7


In [7]:
def execute_pipeline(sex=False, emotion=False, mouth=False):
    """ Put specified files through the pipeline"""
    
    overview = create_csv()
    
    if sex:
        overview = overview[overview['sex'] == sex]
    if emotion:
        overview = overview[overview['emotion'] == emotion]
    if mouth:
        overview = overview[overview['mouth'] == mouth]
    
    files = list(overview['file'])
    
    images = pipe_complex.execute(files, max_depth=1)
    
    return images

In [8]:
# Put selected images through pipeline
execute_pipeline(sex='female', emotion='angry', mouth='open')

[{'ReadImage+ViolaJones_20+CutCircle+FourierFeatures_a8s7': array([[ 6.61933627e+02+5.59481087e+02j, -7.64121496e+03-9.20071563e+02j,
           5.44768627e+02-4.65301215e+02j],
         [ 1.43064935e+02-8.79306016e+01j, -1.04094580e+04+2.39444420e+03j,
          -6.43205553e+01-1.79226316e+02j],
         [-8.46656527e+02-1.20100193e+03j, -8.44889822e+05-4.23483471e-12j,
          -8.46656527e+02+1.20100193e+03j],
         [-6.43205553e+01+1.79226316e+02j, -1.04094580e+04-2.39444420e+03j,
           1.43064935e+02+8.79306016e+01j],
         [ 5.04268627e+02+4.14205716e+02j, -6.93496350e+04+9.20071563e+02j,
           6.21433627e+02-5.08385588e+02j],
         [ 6.47320557e+02+5.42533887e+02j, -4.05295967e+03-3.89740061e+02j,
           4.92917498e+02-7.97578552e+02j],
         [-1.13604305e+03-1.73672772e+03j, -8.57993824e+05+4.09272616e-12j,
          -1.13604305e+03+1.73672772e+03j],
         [ 4.92917498e+02+7.97578552e+02j, -4.05295967e+03+3.89740061e+02j,
           6.47320557e+02-